# 將資料切成 訓練集 & 測試集 (p = 0.1)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import time
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")


In [2]:
path_ = r'C:\Users\e2414\OneDrive\桌面\專題\AllData\training'  #資料從哪裡來


path_final = r'E:\專題\AllData\training\final_training'  #資料去哪

In [21]:
%%time
with open(path_+ r"/train.pkl", "rb") as f:
    train = pickle.load(f)
    
with open(path_+ r"/trainlabel.pkl", "rb") as f:
    trainlabel = pickle.load(f)


Wall time: 26.1 s


In [22]:
print(np.shape(train))
print(np.shape(trainlabel))

sentence_label = list(range(len(trainlabel)))
print(len(sentence_label))


(828845, 300)
(828845,)
828845


<hr>

## 1. 抽所有 828845 筆資料 p = 0.1 當作測試集 (維持資料的 imbalance 分布)

In [23]:
# 抽來當測試的位置
loc = np.random.choice(sentence_label, int(len(sentence_label)*0.1), replace = False)
print(len(loc))

82884


In [24]:
%%time
# 抽來當訓練的位置
left_loc = [i for i in sentence_label if i not in loc]
print(len(left_loc))

745961
Wall time: 40.1 s


In [ ]:
# 訓練會按照順序排，測試不會

In [25]:
%%time
# 測試集 資料
choose_test = train[loc, ]
print(choose_test.shape)

# 測試 label
choose_testlabel = [trainlabel[i] for i in loc]
print(len(choose_testlabel))


# 訓練集 資料
choose_train = train[left_loc, ]
print(choose_train.shape)

# 訓練集 label
choose_trainlabel = [trainlabel[i] for i in left_loc]
print(len(choose_trainlabel))

(82884, 300)
82884
(745961, 300)
745961
Wall time: 976 ms


### 匯出

In [26]:
%%time
# 測試集
with open(path_final + r"/loc_test.pkl",'wb') as f:
    pickle.dump(loc, f)

with open(path_final + r"/X_test.pkl",'wb') as f:
    pickle.dump(choose_test, f)
    
with open(path_final + r"/y_test.pkl",'wb') as f:
    pickle.dump(choose_testlabel, f)
    

# 訓練集
with open(path_final + r"/loc_train.pkl",'wb') as f:
    pickle.dump(left_loc, f)

with open(path_final + r"/X_train.pkl",'wb') as f:
    pickle.dump(choose_train, f)
    
with open(path_final + r"/y_train.pkl",'wb') as f:
    pickle.dump(choose_trainlabel, f)

Wall time: 1min 9s


<hr>

### 在從所有 745961 筆資料中，抽十萬筆當訓練集 (維持資料的 imbalance 分布)

In [ ]:
left_loc
choose_train
choose_trainlabel

In [31]:
alll = list(range(len(left_loc)))

In [32]:
# 抽十萬筆訓練 !
loc = np.random.choice(alll, 100000, replace = False)
print(len(loc))

100000


In [38]:
%%time
rand10_loc = [left_loc[i] for i in loc]
rand10_train = choose_train[loc, ]
rand10_label = [choose_trainlabel[i] for i in loc]

Wall time: 216 ms


In [47]:
print(list(rand10_label.count(i) for i in range(1, 40+1)))

[13306, 12050, 8565, 7838, 7162, 5307, 5444, 4115, 3899, 3410, 2980, 2975, 2759, 1835, 2222, 1387, 1446, 1371, 1174, 1270, 1169, 1037, 1014, 771, 729, 608, 533, 538, 484, 480, 318, 293, 288, 187, 209, 196, 181, 175, 165, 110]


In [45]:
# 訓練集
with open(path_final + r"/loc_train_10im.pkl",'wb') as f:
    pickle.dump(rand10_loc, f)

with open(path_final + r"/X_train_10im.pkl",'wb') as f:
    pickle.dump(rand10_train, f)
    
with open(path_final + r"/y_train_10im.pkl",'wb') as f:
    pickle.dump(rand10_label, f)

### 從以上「已分出」的訓練集抽當訓練

### 從以上「已分出」的測試集抽當測試
(以下沒有使用)

## 2. 每個 group 各抽 1000 筆 (資料是 balance)

In [13]:
def sampling(train, trainlabel, group, sampling):
    
    alldata = list(range(np.shape(train)[0]))  #資料總共幾筆
   
    # 表情 group 對應的位置
    allemo = []
    for j in range(1, group + 1):
        j_emo = [i for i in alldata if trainlabel[i]== j]
        allemo.append(j_emo)
        # allemo:[[group 1 表情符號位置], [group 2 表情符號位置], [], ...[group 40 表情符號位置]]
    
    
    #抽樣
    random_want = []
    for j in range(group):
        gg = np.random.choice(allemo[j], sampling, replace = False).tolist() #每個組隨機抽到的表情位置
        random_want = random_want + gg
        # random_want: [group 1 抽樣的位置, group 2 抽樣的位置 ...]
        # 匯出按照 group排序，label可直接上為 111..11222..22333...33 .....
        
    return allemo, random_want

In [ ]:
# 測試集 資料
choose_test = train[loc, ]
print(choose_test.shape)

# 測試 label
choose_testlabel = [trainlabel[i] for i in loc]
print(len(choose_testlabel))


# 訓練集 資料
choose_train = train[left_loc, ]
print(choose_train.shape)

# 訓練集 label
choose_trainlabel = [trainlabel[i] for i in left_loc]
print(len(choose_trainlabel))

In [11]:
print([choose_trainlabel.count(i) for i in list(range(1, 40+1))])

[99958, 88645, 64403, 58886, 53413, 39396, 40793, 30104, 29578, 25732, 22668, 22814, 20528, 14015, 16423, 10794, 10486, 10416, 9163, 8700, 8571, 7513, 7167, 5673, 5364, 4501, 3974, 3858, 3566, 3452, 2329, 2272, 2006, 1526, 1432, 1438, 1173, 1237, 1102, 892]


In [14]:
%%time
# 40類 800 筆
result = sampling(choose_train, choose_trainlabel, 40, 800)

#抽樣的位置
random_sample = result[1]   # 在已被分出來的 "choose_train" 的位置
print(len(random_sample))

32000
Wall time: 1.7 s


In [102]:
%%time
# 抽到的資料
data1000 = train[random_sample, ]
print(data1000.shape)

# 抽到的資料 label
datalabel1000 = [trainlabel[i] for i in random_sample]
print(len(datalabel1000))

(40000, 300)
40000
Wall time: 2.55 s


## 將資料切成 train、test

In [121]:
# 抽來當 test 的位置 (在 sampling 中的位置)
loc = np.random.choice(list(range(len(datalabel1000))), int(len(datalabel1000)*0.1), replace = False)

# 抽來當 test 的位置 (在 實際 85 萬句向量中的位置)
sen_test_loc = [random_sample[i] for i in loc]
print(len(sen_test_loc))

4000


In [123]:
%%time

# 抽來當 training 的位置 (在 sampling 中的位置)
left_loc = [i for i in list(range(len(datalabel1000))) if i not in loc]

# 抽來當 training 的位置 (在 實際 85 萬句向量中的位置)
sen_train_loc = [random_sample[i] for i in left_loc]
print(len(sen_test_loc))

36000
Wall time: 203 ms


In [125]:
%%time
# 測試集 資料
choose_test = data1000[loc, ]
print(choose_test.shape)

# 測試 label
choose_testlabel = [datalabel1000[i] for i in loc]
print(len(choose_testlabel))


# 訓練集 資料
choose_train = data1000[left_loc, ]
print(choose_train.shape)

# 訓練集 label
choose_trainlabel = [datalabel1000[i] for i in left_loc]
print(len(choose_trainlabel))

(4000, 300)
4000
(36000, 300)
36000
Wall time: 50.5 ms


## 匯出

In [86]:
%%time
# 測試集
with open(path_final + r"/loc_test_s1000.pkl",'wb') as f:
    pickle.dump(sen_test_loc, f)

with open(path_final + r"/X_test_s1000.pkl",'wb') as f:
    pickle.dump(test1000, f)
    
with open(path_final + r"/y_test_s1000.pkl",'wb') as f:
    pickle.dump(test_label1000, f)
    
    
# 訓練集
with open(path_final + r"/loc_train_s1000.pkl",'wb') as f:
    pickle.dump(sen_train_loc, f)

with open(path_final + r"/X_train.pkl_s1000",'wb') as f:
    pickle.dump(choose_train, f)
    
with open(path_final + r"/y_train_s1000.pkl",'wb') as f:
    pickle.dump(choose_trainlabel, f)

Wall time: 5.29 s
